In [1]:
import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
read_data = pd.read_csv("shopping-ca-cls.data.tsv", 
                        encoding="utf-8", names=['shoes_species', 'maker_name', 'brand_name', 'price','image_url','name'],
                        sep='\t')
# test data 읽어오기.

read_test_data = pd.read_csv("shopping-ca-cls.data.test.tsv", 
                        encoding="utf-8", names=['shoes_species', 'maker_name', 'brand_name', 'price','image_url','name'],
                        sep='\t')

In [ ]:
from konlpy.tag import Twitter
twitter = Twitter()


def nlp_with_konolpy_twitter_class():
    print('konolpy load complete')


# 특수 문자제거 , 자연어 처리
def process_character(name):
    # 특수 문자제거
    new_name = re.sub("[\,\.\%\-\_\/]",' ',name)
    #new_name = re.sub("[^\D]",' ', new_name)
    #new_name = re.sub("[\d]",' ', new_name)
    new_name = new_name.strip()
    
    # 자연어 처리
    nouns_str = twitter.morphs(new_name)
    make_new_str = ''
    for r in nouns_str:
        make_new_str =  make_new_str + r + ' '
    
    #print(make_new_str)   
    #twitter.nouns
    #print(new_name)
    return make_new_str
    
    
def test_from_input_product_name():
    while (1) :
        input_product_name = input()
        pred = clf.predict(vectorizer.transform([input_product_name]))[0]
        print("예상되는 상품태그는 : " , list(category_dict.keys())[list(category_dict.values()).index(pred)])

In [ ]:
# todo 새로운 상품 이름 만든다음에 단어추가!
shoes_category_list = []
new_shoes_name_for_vectornizer_list = []
count = 0
for each_row in read_data.iterrows():
    #print(each_row[1]['name'])
    processed_name = process_character(each_row[1]['name'])
    #print(each_row[1]['maker_name'])
    #print(each_row[1]['brand_name'])
    if pd.isnull(each_row[1]['maker_name']) == False :
        processed_name = processed_name + each_row[1]['maker_name'] + ' '
    if pd.isnull(each_row[1]['brand_name']) == False :
        processed_name = processed_name + each_row[1]['brand_name']
    
    new_shoes_name_for_vectornizer_list.append(processed_name)
    shoes_category_list.append(each_row[1]['shoes_species'])
    #new_shoes_name_for_vectornizer_list.append(each_row[1]['name'])
    
    
vectorizer = CountVectorizer()
x_list = vectorizer.fit_transform(new_shoes_name_for_vectornizer_list)
#print(x_list)

y_list = []
category_dict = dict(zip(list(set(shoes_category_list)),range(len(set(shoes_category_list)))))

print(category_dict)
for each in read_data.iterrows():
    y_list.append(category_dict[each[1]['shoes_species']])
    
#print(len(set(shoes_category_list)))
#print(category_dict[u'기능화']

In [ ]:
from sklearn import svm, grid_search
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
import numpy as np

svc_param = {'C':np.logspace(-2,0,20)}
grid_search_cv = GridSearchCV(svm.LinearSVC(),param_grid,cv=5,n_jobs=4)
grid_search_cv.fit(x_list,y_list)
print(grid_search_cv.best_params_, grid_search_cv.best_score_)

#tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
#                     'C': [1, 10, 100, 1000]},
#                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]
#

#Cs = [0.001, 0.01, 0.1, 1, 10]
#gammas = [0.001, 0.01, 0.1, 1]
#param_grid = {'C': Cs, 'gamma' : gammas}
# tuned_parameters = {'C':np.logspace(-2,0,20)}
# 
# Set the parameters by cross-validation
#tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
#                     'C': [1, 10, 100, 1000]},
#                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]
#clf = GridSearchCV(SVC(C=1), tuned_parameters, cv=5, scoring=score)
#
#grid_search_cv2 = GridSearchCV(svm.SVC(),param_grid,cv=5,n_jobs=4)
3
#grid_search_cv.fit(x_list,y_list)
#grid_search_cv2.fit(x_list,y_list)

#print(grid_search_cv.best_params_, grid_search_cv.best_score_);
#print(grid_search_cv2.best_params_, grid_search_cv2.best_score_);

#from sklearn.grid_search import GridSearchCV

In [ ]:
clf = svm.LinearSVC(C=grid_search_cv.best_params_['C'])
clf.fit(x_list,y_list)

count_num = 0
yes_num = 0

pred = clf.predict(vectorizer.transform(['안전화_WK504NR_265mm_|조| 안전화 작업화 경량안전화 소방 안전용품']))[0]
print(pred);
print(y_list[0])

#print(list(category_dict.keys())[list(category_dict.values()).index(pred)]);

for each_row in read_data.iterrows():
   # predict_x = vectorizer.transform(each_row[1]['name'])
    #print(clf.predict(vectorizer.transform([each_row[1]['name']]))[0])
    #print(y_list[count_num])
    processed_name = process_character(each_row[1]['name'])
    
    if pd.isnull(each_row[1]['maker_name']) == False :
        processed_name = processed_name + each_row[1]['maker_name'] + ' '
    if pd.isnull(each_row[1]['brand_name']) == False :
        processed_name = processed_name + each_row[1]['brand_name']
        
    if clf.predict(vectorizer.transform([processed_name]))[0] == y_list[count_num] :
        yes_num = yes_num + 1
        
    count_num = count_num + 1
    
print('Accuracy Score : ', yes_num / count_num)

#print(count_num)
#print(yes_num)
#print("yes : %f", yes_num / count );

In [ ]:
# 상품 이름만 처리 하였을때 -> 원본 데이터와 일치율 정답율 : 0.9720026835534175 테스트 데이터로 모델테스트 61%
# 상품 이름 + 특수 문자제거 -> 원본 데이터와 일치율 0.9140117487277666 테스트 데이터로 모델테스트 ,0.6118598382749326 \
# 상품 이름 + 특수문자제거 + NLP 처리. -> 원본 데이터와 일치율 0.67 테스트 데이터로 모델테스트 66%

In [ ]:
y_list2 = []
yes_num2 = 0
count_num2 = 0

for each in read_test_data.iterrows():
    shoes_species_dict = category_dict[each[1]['shoes_species']] 
    
    #original_dict = clf.predict(vectorizer.transform([each[1]['name']]))[0]
    #print(shoes_species_dict,original_dict)
    processed_test_data_name = process_character(each[1]['name'])
    if pd.isnull(each[1]['maker_name']) == False :
        processed_test_data_name = processed_test_data_name + each[1]['maker_name'] + ' '
    if pd.isnull(each[1]['brand_name']) == False :
        processed_test_data_name = processed_test_data_name + each[1]['brand_name']
        
    if clf.predict(vectorizer.transform([processed_test_data_name]))[0] == shoes_species_dict :
        yes_num2 = yes_num2 + 1
        
    count_num2 = count_num2 + 1
    
#print(yes_num2, count_num2)    
print('Accuracy Score : ', yes_num2 / count_num2)